# Zero to Hero

## 1.12 Funcion de Ganancia de Arbol training/testing

El objetivo es poner el notebook anterior dentro de una funcion, e invocarla con distintas semillas, para maravillarnos ante la dispersion de las ganancias

In [36]:
rm(list = ls())
gc()

library("data.table") # cargo la libreria  data.table
library("rpart") # cargo la libreria  rpart
library("rpart.plot")
library("caret")

options(repr.plot.width = 20, repr.plot.height = 10)
setwd("~/buckets/b1/")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2386989,127.5,4768802,254.7,3145080,168.0
Vcells,4174098,31.9,198651557,1515.6,248256670,1894.1


In [37]:
GananciaArbol <- function(semilla, data, x, train = 0.70) {
  # establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(datajulio$clase_ternaria, p = 0.70, list = FALSE)

  modelo <- rpart::rpart(
    formula = "clase_ternaria ~ .",
    data = data[train_rows], # los datos de training
    xval = 0,
    control = x
  )

  # Aplico el modelo a los datos de testing
  prediccion <- predict(modelo, data[-train_rows], type = "prob")

  prob_baja2 <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[-train_rows, sum((prob_baja2 > 0.025) * ifelse(clase_ternaria == "BAJA+2", 117000, -3000))]

  return(ganancia_testing)
}


Vale la pena notar que dentro de la funcion no se han creado  dtrain y dtest,  sino que directamente se utilizan:
* data[ train_rows]  como training
* data[ -train_rows]  como testing


#Aqui empieza el programa

In [38]:
dataset <- fread("./datasets/dataset_pequeno.csv") # cargo el dataset #cargo el dataset
datajulio <- dataset[foto_mes == 202107] # me quedo solo con el periodo 202107
datasept <- dataset[foto_mes == 202109]


In [39]:
# defino unos buenos hiperparametros
param <- list(
    "cp" = -0.5,
    "minsplit" = 500,
    "minbucket" = 10,
    "maxdepth" = 5
)


In [40]:
# Ahora hago algunas llamadas a la funcion


In [41]:
GananciaArbol(11, datajulio, x = param)


[1] 15618000

In [42]:
prediccion <- predict(modelo, datasept, type = "prob")

prob_baja2 <- prediccion[, "BAJA+2"]
Predicted <- ifelse(prob_baja2 > 0.025, 1, 0) 

entrega <- as.data.table(list("numero_de_cliente" = datasept$numero_de_cliente, "Predicted" = Predicted)) #creo tabla

ERROR: Error in eval(expr, envir, enclos): object 'modelo' not found
